> **Rappel** : clique sur une cellule grise, puis **Shift + Entree** pour l'executer.
> Execute les cellules **dans l'ordre** de haut en bas.

---

# Lecon 4 : L'attention

## L'ingredient secret des GPT

Imagine que tu lis la phrase : "Le **chat** noir dort sur le **canape**."

Si on te demande "Ou dort le chat ?", ton cerveau fait automatiquement
le lien entre "dort", "chat" et "canape" -- meme si ces mots ne sont
pas cote a cote.

C'est exactement ce que fait le **mecanisme d'attention** :
il permet au modele de regarder **n'importe quel** element du passe,
pas seulement les derniers.

In [ ]:
# ============================================================
# Cellule d'initialisation \u2014 execute sans lire (Shift+Entree)
# ============================================================

import json
import uuid

from IPython.display import HTML, display

_exercices_faits = set()
_NB_TOTAL = 2


def verifier(num_exercice, condition, message_ok, message_aide=""):
    """Valide un exercice avec feedback HTML vert/rouge + compteur."""
    try:
        _result = bool(condition)
    except Exception:
        _result = False
    if _result:
        _exercices_faits.add(num_exercice)
        n = len(_exercices_faits)
        barre = "\U0001f7e9" * n + "\u2b1c" * (_NB_TOTAL - n)
        display(
            HTML(
                f'<div style="padding:10px;background:#d4edda;border-left:5px solid #28a745;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\u2705 <b>{message_ok}</b><br>"
                f'<span style="color:#555">Progression : {barre} {n}/{_NB_TOTAL}</span></div>'
            )
        )
        if n == _NB_TOTAL:
            display(
                HTML(
                    '<div style="padding:12px;background:linear-gradient(135deg,#3949ab,#6a1b9a);'
                    "color:white;border-radius:8px;text-align:center;font-family:system-ui,-apple-system,sans-serif;"
                    'font-size:1.2em;margin:8px 0">\U0001f3c6 <b>Bravo ! Toutes les activites de cette lecon sont terminees !</b></div>'
                )
            )
    else:
        display(
            HTML(
                f'<div style="padding:10px;background:#fff3cd;border-left:5px solid #ffc107;'
                f'margin:8px 0;border-radius:4px;font-family:system-ui,-apple-system,sans-serif">'
                f"\U0001f4a1 <b>{message_aide}</b></div>"
            )
        )


def exercice(numero, titre, consigne, observation=""):
    """Affiche la banniere d'exercice."""

    def _style_code(text):
        return text.replace(
            "<code>",
            '<code style="font-size:0.95em;background:#bbdefb;'
            'padding:1px 5px;border-radius:3px;font-family:monospace;">',
        )

    obs = ""
    if observation:
        obs = (
            f'<div style="margin-top:6px;color:#555;font-size:0.92em;">'
            f"<b>Ce que tu vas voir\u00a0:</b> {_style_code(observation)}</div>"
        )
    display(
        HTML(
            f'<div style="border-left:5px solid #1565c0;background:#e8f0fe;'
            f"padding:12px 16px; margin:4px 0 10px 0; border-radius:0 8px 8px 0;"
            f'font-family:system-ui,-apple-system,sans-serif; font-size:0.95em;">'
            f'<b style="color:#0d47a1;">Exercice\u00a0{numero} \u2014 {titre}</b><br>'
            f"{_style_code(consigne)}{obs}</div>"
        )
    )


def afficher_attention(poids, positions, titre="Poids d'attention"):
    """Attention BertViz-style : lignes entre tokens, opacite = poids."""
    n = len(positions)
    col_w, row_h = 60, 36
    svg_w = col_w * n + 40
    svg_h = row_h * 3 + 20
    # Tokens en haut (Keys) et token query en bas
    elems = ""
    for i, c in enumerate(positions):
        x = 20 + i * col_w + col_w // 2
        # Token label en haut
        elems += (
            f'<text x="{x}" y="20" text-anchor="middle" '
            f'font-size="16" font-weight="bold" fill="#333">{c}</text>'
        )
        # Ligne de connexion : opacite = poids d'attention
        w = poids[i]
        opacity = max(w, 0.05)
        stroke_w = 1 + w * 8  # epaisseur proportionnelle
        elems += (
            f'<line x1="{x}" y1="28" x2="{svg_w // 2}" y2="{svg_h - 30}" '
            f'stroke="#1565c0" stroke-width="{stroke_w:.1f}" '
            f'stroke-opacity="{opacity:.2f}" stroke-linecap="round"/>'
        )
        # Poids en pourcentage sous le token
        elems += (
            f'<text x="{x}" y="42" text-anchor="middle" '
            f'font-size="11" fill="#555">{w:.0%}</text>'
        )
    # Label "query" en bas
    elems += (
        f'<text x="{svg_w // 2}" y="{svg_h - 10}" text-anchor="middle" '
        f'font-size="14" font-weight="bold" fill="#1565c0">query</text>'
    )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<svg width="{svg_w}" height="{svg_h}" '
            f'style="margin-top:4px;display:block">{elems}</svg></div>'
        )
    )


def afficher_masque_causal(mot, titre="Masque causal"):
    """Masque causal interactif : clic sur une position = highlight ce qu'elle peut voir."""
    uid = uuid.uuid4().hex[:8]
    n = len(mot)
    lettres_js = json.dumps(list(mot))
    # Construire le tableau HTML
    header = "".join(
        f'<th style="padding:6px 10px;font-size:1.1em">{c}</th>' for c in mot
    )
    rows = ""
    for i, c in enumerate(mot):
        row_cells = ""
        for j in range(n):
            if j <= i:
                row_cells += (
                    f'<td data-r="{i}" data-c="{j}" style="padding:6px 10px;'
                    f"background:#d4edda;text-align:center;border:1px solid #ccc;"
                    f'cursor:pointer">\u2705</td>'
                )
            else:
                row_cells += (
                    f'<td data-r="{i}" data-c="{j}" style="padding:6px 10px;'
                    f"background:#f8d7da;text-align:center;border:1px solid #ccc;"
                    f'cursor:pointer">\u274c</td>'
                )
        rows += (
            f'<tr><th style="padding:6px 10px;font-size:1.1em">{c}</th>{row_cells}</tr>'
        )
    display(
        HTML(
            f"<!-- tuto-viz -->"
            f'<div style="margin:8px 0"><b>{titre}</b>'
            f'<table id="m{uid}" style="border-collapse:collapse;margin-top:4px">'
            f"<tr><th></th>{header}</tr>{rows}</table>"
            f'<div id="mi{uid}" style="margin-top:4px;color:#555;font-size:0.9em;min-height:1.3em">'
            f"Clique sur une ligne pour voir ce que cette lettre peut regarder</div></div>"
            f'<script>(function(){{"use strict";'
            f'var t=document.getElementById("m{uid}"),'
            f'info=document.getElementById("mi{uid}"),'
            f"L={lettres_js};"
            f't.addEventListener("click",function(e){{'
            f'var th=e.target.closest("th");'
            f'var td=e.target.closest("td[data-r]");'
            f"if(!th&&!td)return;"
            f"var r=td?+td.dataset.r:null;"
            f"if(th){{var tr=th.parentElement;r=[].indexOf.call(tr.parentElement.children,tr)-1}}"
            f"if(r<0||r>=L.length)return;"
            f't.querySelectorAll("td[data-r]").forEach(function(d){{'
            f'd.style.outline="";d.style.fontWeight=""}});'
            f't.querySelectorAll("td[data-r=\'" +r+"\']" ).forEach(function(d){{'
            f'd.style.outline="2px solid #1565c0";d.style.fontWeight="bold"}});'
            f"var visible=[];for(var j=0;j<=r;j++)visible.push(L[j]);"
            f'info.textContent=L[r]+" peut voir : ["+visible.join(", ")+"]"'
            f"}})"
            f"}})();</script>"
        )
    )


print("Outils de visualisation charges !")

---
## Comment ca marche ?

Pour chaque lettre, le modele se pose 3 questions :

1. **Query (Q)** : "Qu'est-ce que je cherche ?" (ce que cette lettre a besoin de savoir)
2. **Key (K)** : "Qu'est-ce que j'offre ?" (ce que cette lettre peut apporter)
3. **Value (V)** : "Quelle info je transmets ?" (l'information reelle)

L'attention = comparer chaque Query avec toutes les Keys pour trouver
les lettres les plus utiles, puis collecter leurs Values.

Prenons le mot **"chat"** comme exemple. Chaque lettre a un
**embedding** (un petit vecteur de 4 nombres) :

In [ ]:
import math
import random

random.seed(42)

# Notre mot d'exemple
mot = "chat"
print(f"Mot : '{mot}'")
print(f"Positions : {list(enumerate(mot))}")
print()

# Chaque lettre a un embedding (simplifie a 4 dimensions)
DIM = 4
emb = {
    "c": [1.0, 0.0, 0.5, -0.3],  # embedding de 'c'
    "h": [0.2, 0.8, -0.1, 0.5],  # embedding de 'h'
    "a": [0.5, 0.3, 0.9, 0.1],  # embedding de 'a'
    "t": [-0.3, 0.6, 0.2, 0.8],  # embedding de 't'
}

for c, v in emb.items():
    print(f"  '{c}' -> {v}")  # 4 nombres par lettre

### Calculer les scores d'attention

Calculons l'attention pour la lettre **"t"** (derniere position).
Question : quelles lettres precedentes sont importantes pour predire
ce qui vient apres "t" dans "chat" ?

In [ ]:
# Pour simplifier, on utilise les embeddings directement comme Q, K, V
# (en vrai, il y a des matrices de transformation)


def produit_scalaire(a, b):
    """Mesure la similarite entre deux vecteurs."""
    return sum(x * y for x, y in zip(a, b, strict=False))


def softmax(scores):
    """Transforme les scores en probabilites."""
    max_s = max(scores)
    exps = [math.exp(s - max_s) for s in scores]
    total = sum(exps)
    return [e / total for e in exps]


# La "query" de 't' = son embedding
query = emb["t"]  # Ce que 't' cherche

# Comparer avec toutes les lettres (y compris elle-meme)
scores = []
for c in mot:
    key = emb[c]  # Ce que chaque lettre offre
    # Produit scalaire puis division par sqrt(DIM)
    # On divise par sqrt(DIM) pour eviter des scores trop grands
    score = produit_scalaire(query, key) / math.sqrt(DIM)
    scores.append(score)

print("Scores d'attention pour 't' :")
for c, s in zip(mot, scores, strict=False):
    print(f"  '{c}' : {s:.2f}")

# Transformer en probabilites avec softmax
poids_attention = softmax(scores)
print()
print("Poids d'attention (apres softmax) :")
for c, w in zip(mot, poids_attention, strict=False):
    barre = "#" * int(w * 30)  # barre proportionnelle
    print(f"  '{c}' : {w:.1%} {barre}")

print()
print("Le modele 'regarde' plus les lettres avec un poids eleve !")

# Visualisation BertViz-style : lignes entre tokens
afficher_attention(
    poids_attention, list(mot), titre="Poids d'attention pour 't' dans 'chat'"
)

In [ ]:
exercice(
    1,
    "Change la lettre qui pose la question",
    'Change <code>ma_lettre</code> ci-dessous, puis <b>Shift + Entree</b> (essaie <code>"c"</code>, <code>"h"</code> ou <code>"a"</code>).',
    "Les scores d'attention changent : chaque lettre 'regarde' differemment.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

ma_lettre = "t"  # <-- Essaie "c", "h" ou "a" !

# ╚══════════════════════════════════════╝

if ma_lettre not in emb:
    print(f"Erreur : '{ma_lettre}' n'est pas dans le mot 'chat'.")
    print("Choisis c, h, a ou t.")
else:
    query_test = emb[ma_lettre]
    scores_test = []
    for c in mot:
        key = emb[c]
        score = produit_scalaire(query_test, key) / math.sqrt(DIM)
        scores_test.append(score)
    poids_test = softmax(scores_test)
    print(f"Poids d'attention quand '{ma_lettre}' pose la question :")
    for c, w in zip(mot, poids_test, strict=False):
        barre = "#" * int(w * 30)
        print(f"  '{c}' : {w:.1%} {barre}")

    # Visualisation BertViz-style
    afficher_attention(
        poids_test,
        list(mot),
        titre=f"Poids d'attention quand '{ma_lettre}' pose la question",
    )

# Validation
verifier(
    1,
    ma_lettre != "t",
    "Bravo ! Tu as change la lettre qui pose la question.",
    "Change ma_lettre pour une autre lettre du mot 'chat' (c, h ou a).",
)

### Collecter l'information (les Values)

L'attention a choisi les lettres importantes. Maintenant on collecte
leur information : on fait une **somme ponderee** des Values.

Chaque Value est multipliee par son poids d'attention, puis on additionne :

In [ ]:
# Somme ponderee des Values : chaque lettre contribue
# proportionnellement a son poids d'attention
resultat = [0.0] * DIM  # vecteur resultat (meme taille que les embeddings)

for c, w in zip(mot, poids_attention, strict=False):
    value = emb[c]  # l'information de cette lettre
    for d in range(DIM):
        resultat[d] += w * value[d]  # poids x valeur

print("Vecteur de sortie de l'attention :")
print(f"  {[f'{x:.2f}' for x in resultat]}")
print()
print("Ce vecteur combine l'information de toutes les lettres,")
print("en donnant plus de poids aux lettres les plus pertinentes.")
print()
print("C'est cette information qui sera utilisee pour predire")
print("la lettre suivante !")

**Qu'est-ce que tu remarques ?**

Le resultat combine les infos de toutes les lettres.
Lesquelles ont le plus contribue ? Regarde les poids d'attention
ci-dessus pour trouver la reponse !

---

## Attention causale : pas de triche !

Regle importante : quand le modele predit la lettre suivante,
il **ne peut pas regarder le futur** -- seulement le passe.

```
Pour predire apres 'c' : peut regarder [c]
Pour predire apres 'h' : peut regarder [c, h]
Pour predire apres 'a' : peut regarder [c, h, a]
Pour predire apres 't' : peut regarder [c, h, a, t]
```

On appelle ca le **masque causal**. C'est ce qui fait de GPT un modele
**auto-regressif** : il genere un token a la fois, de gauche a droite.

**Clique sur une ligne** du tableau pour voir ce que chaque lettre peut regarder :

In [ ]:
# Affichons le masque causal en texte
print("Masque causal pour 'chat' :")
print()
print("          c    h    a    t")
for i, c in enumerate(mot):
    row = ""
    for j in range(len(mot)):
        if j <= i:  # peut regarder (passe + soi-meme)
            row += "  OK "
        else:  # interdit (futur)
            row += "  -- "
    print(f"  {c} : {row}")

print()
print("OK = peut regarder, -- = interdit (c'est le futur)")

# Masque causal interactif (clique sur une ligne !)
afficher_masque_causal(mot, titre="Masque causal pour 'chat'")

In [ ]:
exercice(
    2,
    "Change le mot",
    'Change <code>mon_mot</code> ci-dessous (essaie <code>"plume"</code>, <code>"arbre"</code> ou ton prenom).',
    "Le masque s'adapte a la longueur du mot.",
)

# ╔══════════════════════════════════════╗
# ║  A TOI DE JOUER !                    ║
# ╠══════════════════════════════════════╣

mon_mot = "chat"  # <-- Essaie "plume", "arbre" ou ton prenom !

# ╚══════════════════════════════════════╝

print(f"Masque causal pour '{mon_mot}' :")
print()
header = "     " + "".join(f"  {c}  " for c in mon_mot)
print(header)
for i, c in enumerate(mon_mot):
    row = ""
    for j in range(len(mon_mot)):
        if j <= i:
            row += "  OK "
        else:
            row += "  -- "
    print(f"  {c} : {row}")
print()
print("Chaque lettre ne voit que les lettres avant elle (et elle-meme).")
print(f"Le triangle grandit avec la longueur du mot ({len(mon_mot)} lettres ici).")

# Masque causal interactif
afficher_masque_causal(mon_mot, titre=f"Masque causal pour '{mon_mot}'")

# Validation
verifier(
    2,
    mon_mot != "chat",
    "Super ! Tu as explore le masque causal d'un autre mot.",
    "Change mon_mot pour un autre mot, par exemple 'plume' ou 'arbre'.",
)

---
## Multi-tetes : regarder de plusieurs facons

En pratique, on utilise **plusieurs tetes d'attention** en parallele.
Comme si **4 eleves posent chacun une question differente** :

- Tete 1 : quelles lettres forment des syllabes ensemble ?
- Tete 2 : quelle est la voyelle la plus recente ?
- Tete 3 : est-ce que c'est un debut ou une fin de mot ?
- Tete 4 : quelle lettre est la plus rare ?

Les resultats de toutes les tetes sont combines pour une prediction finale.

Dans microgpt.py de Karpathy, il y a **4 tetes** d'attention.

---
## Resume visuel

```
Lettres d'entree :  [c] [h] [a] [t]
                     |   |   |   |
                     v   v   v   v
Embeddings :        [---] [---] [---] [---]
                     |   |   |   |
                     v   v   v   v
Attention :         Chaque lettre regarde les precedentes
                    et collecte l'info importante
                     |   |   |   |
                     v   v   v   v
Prediction :        Quelle est la prochaine lettre ?
```

---
## Ce qu'on a appris

- L'**attention** permet au modele de regarder toutes les lettres precedentes, pas juste la derniere
- Ca fonctionne avec **Q** (je cherche), **K** (j'offre), **V** (mon info)
- Le **masque causal** empeche de tricher en regardant le futur
- Plusieurs **tetes** d'attention regardent des choses differentes en parallele

### Derniere etape

On a maintenant toutes les pieces du puzzle. Dans la prochaine lecon,
on assemble tout pour construire un vrai mini-LLM !

---
*Prochaine lecon : [05 - Mon premier LLM](05_mon_premier_llm.ipynb)*

---

### Sources (ISO 42001)

- **Mecanisme Q/K/V et masque causal** : [microgpt.py](https://gist.github.com/karpathy/8627fe009c40f57531cb18360106ce95) \u2014 Andrej Karpathy, section self-attention
- **"Attention Is All You Need"** : Vaswani et al., 2017, [arXiv:1706.03762](https://arxiv.org/abs/1706.03762) \u2014 article fondateur des Transformers
- **Explication visuelle de l'attention** : [Video "Let's build GPT"](https://www.youtube.com/watch?v=kCc8FmEb1nY) \u2014 Andrej Karpathy (2023), section attention
- **Multi-head attention (4 tetes dans microgpt.py)** : meme source, parametre `n_head=4`